### MONGODB ATLAS 내담자정보 임베딩 생성(거주지역, 국가, 체류자격, 기타정보)

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from langchain_mongodb import MongoDBAtlasVectorSearch  # 임포트 경로 변경
from langchain_openai import OpenAIEmbeddings

In [5]:
from pymongo import MongoClient
from openai import OpenAI
import time
from tqdm import tqdm  # 진행상황 표시용
from pathlib import Path
from dotenv import load_dotenv
import os

# 절대 경로 지정
env_path = Path('..') / '.env'
load_dotenv(env_path)

True

In [11]:
# OpenAI 클라이언트 설정
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# MongoDB 연결
mongo_client = MongoClient(os.getenv('MONGODB_ATLAS_CLUSTER_URI'))
db = mongo_client['HelloWorld-AI']
collection = db['foreigner_legal_test']

In [12]:
def create_embedding(text):
    """텍스트를 임베딩 벡터로 변환"""
    try:
        response = client.embeddings.create(
            model="text-embedding-3-large",
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 중 에러 발생: {e}")
        return None

In [13]:
def process_documents():
    """모든 문서를 처리하고 임베딩 추가"""
    # 전체 문서 가져오기
    documents = list(collection.find({}))
    print(f"총 {len(documents)}개의 문서를 처리합니다.")

    # 처리된 문서 수 추적
    success_count = 0
    error_count = 0

    # tqdm으로 진행상황 표시
    for doc in tqdm(documents, desc="문서 처리 중"):
        try:
            # 내담자 정보를 텍스트로 변환
            info_text = f"""
            거주지역: {doc['내담자_정보'].get('거주지역', '')}
            국적: {doc['내담자_정보'].get('국적', '')}
            체류자격: {doc['내담자_정보'].get('체류자격', '')}
            추가정보: {doc['내담자_정보'].get('추가정보', '')}
            """.strip()

            # 임베딩 생성
            embedding = create_embedding(info_text)
            
            if embedding:
                # 문서 업데이트
                collection.update_one(
                    {'_id': doc['_id']},
                    {
                        '$set': {
                            '내담자_정보.raw_text': info_text,
                            '내담자_정보.Embedding': embedding
                        }
                    }
                )
                success_count += 1
                
                # OpenAI API 속도 제한 고려
                time.sleep(0.5)  # 0.5초 대기
            
        except Exception as e:
            print(f"\n문서 {doc['_id']} 처리 중 에러 발생: {e}")
            error_count += 1

    return success_count, error_count

In [14]:
def main():
    try:
        print("임베딩 처리 시작...")
        success, errors = process_documents()
        print(f"\n처리 완료!")
        print(f"성공: {success}개")
        print(f"실패: {errors}개")
        
    except Exception as e:
        print(f"실행 중 에러 발생: {e}")
    
    finally:
        # MongoDB 연결 종료
        mongo_client.close()

if __name__ == "__main__":
    main()

임베딩 처리 시작...
총 249개의 문서를 처리합니다.


문서 처리 중: 100%|██████████| 249/249 [04:33<00:00,  1.10s/it]


처리 완료!
성공: 249개
실패: 0개


In [2]:
# 절대 경로 지정
env_path = Path('..') / '.env'
load_dotenv(env_path)

# MongoDB Atlas 연결
client = MongoClient(os.getenv("MONGODB_ATLAS_CLUSTER_URI"), server_api=ServerApi('1'))

# MongoDB 컬렉션 설정
MONGODB_COLLECTION = client["HelloWorld-AI"]["foreigner_legal_test"]

# Vector Search 설정
db = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    index_name="vector_index",
    relevance_score_fn="cosine"
)

In [26]:
# 첫 문서 가져오기
document = MONGODB_COLLECTION.find_one()
print("문서 구조:")
for key, value in document.items():
    print(f"{key}: {type(value)}")

print("\n  내담자 정보:")
print(json.dumps(document['내담자_정보'], ensure_ascii=False, indent=2))
# print(document)

문서 구조:
_id: <class 'bson.objectid.ObjectId'>
상담제목: <class 'dict'>
상담내용요약: <class 'dict'>
진행_과정_및_결과: <class 'str'>
내담자_정보: <class 'dict'>
해결방법: <class 'str'>
metadata: <class 'dict'>

  내담자 정보:
{
  "거주지역": "여주시",
  "국적": "우즈베키스탄",
  "체류자격": "E-9",
  "추가정보": "\n중도 퇴직한 외국인 근로자\n2020년 종합소득세 체납으로 고용허가 연장 불가능했던 상황\n현재까지 체류자격 연장이 되지 않아 불안함\n체류자격은 당장 곧 만료 예정\n"
}


# MONGODB Vector Search 생성

MONGODB ATLAS Ui 활용

In [29]:
# from pymongo import SearchIndexModel

# # Vector Search 인덱스 설정
# index_definition = {
#     "mappings": {
#         "dynamic": True,
#         "fields": {
#             "내담자_정보": {
#                 "type": "document",
#                 "fields": {
#                     "vector": {
#                         "type": "knnVector",
#                         "dimensions": 1536,  # OpenAI의 embedding 차원
#                         "similarity": "cosine"
#                     }
#                 }
#             }
#         }
#     }
# }

# # SearchIndexModel 생성
# search_index = SearchIndexModel(
#     definition=index_definition,
#     name="내담자_vector_index"
# )

# # 인덱스 생성
# MONGODB_COLLECTION.create_search_index(search_index)

# # 생성된 인덱스 확인
# indexes = MONGODB_COLLECTION.list_search_indexes()
# for index in indexes:
#     print(index)

# 내담자 정보 유사도 계산

In [42]:
def get_similar_cases(user_info: dict, top_k: int = 3):
    """유저 정보와 가장 유사한 상담 사례 찾기"""
    
    # 검색 쿼리 생성
    query_text = f"{user_info.get('현재 처한 상황', '')} {user_info.get('국적', '')} {user_info.get('체류자격', '')}"
    
    pipeline = [
        {
            "$search": {
                "index": "clientinfo_vector_index",
                "text": {
                    "query": query_text,
                    "path": {
                        "wildcard": "*"
                    }
                }
            }
        },
        {
            "$limit": top_k
        },
        {
            "$project": {
                "score": { "$meta": "searchScore" },
                "내담자_정보": 1,
                "상담제목": 1,
                "상담내용요약": 1,
                "해결방법": 1
            }
        }
    ]
    
    results = list(MONGODB_COLLECTION.aggregate(pipeline))
    print(f"Found {len(results)} results")
    return results

# 테스트
user_info = {
    # "거주지역": "경기도 안산시",
    # "국적": "베트남",
    "체류자격": "E-9",
    "현재 처한 상황": "3년 전 한국인 남자와 결혼, 가정 폭력이 빈번히 발생해 생활이 위태로운 상황"
}

similar_cases = get_similar_cases(user_info)
for idx, doc in enumerate(similar_cases, 1):
    print(f"\n{idx}번째 유사 사례:")
    print(f"유사도 점수: {doc.get('score', 0):.4f}")
    print(f"내담자 정보: {doc.get('내담자_정보', {})}")
    # print(f"상담 제목: {doc.get('상담제목', '')}")
    print(f"해결 방법: {doc.get('해결방법', '')}")

Found 3 results

1번째 유사 사례:
유사도 점수: 7.3025
내담자 정보: {'거주지역': '광주광역시', '국적': '우즈베키스탄', '체류자격': 'F-4', '추가정보': '\n국민연금 외국인 가입자 관련 및 대상자 안내 문의\n우즈베키스탄 고용노동부 한국주재사무소 담당자와 상담\n'}
해결 방법: 

외국인 노동자의 국민연금 가입 및 수령에 관한 정보를 제공하였습니다. 외국인 사업장 가입자 및 지역 가입자를 대상으로, 국민연금 가입 대상 및 요건에 대한 안내를 진행하였습니다. 우즈베키스탄은 보험료 면제 협정에 따라 E-8, E-9 체류자격자는 협정가입증명 없이 면제된다는 점을 설명하였습니다. 또한, 외국인 노동자가 국민연금 회원으로 가입하고 반환일시금을 공항에서 지급받는 방법을 안내하였습니다. 국민연금 공단을 직접 방문해 제출해야 할 서류와, 인천공항 상담센터에서의 절차도 설명하였습니다. 우즈베키스탄 담당 기관과의 협력을 통해, 국민연금 가입 당연 국가가 아닌 경우 수령 확답이 어려운 점을 연구하고 해결 방안을 도출하기로 하였습니다.


2번째 유사 사례:
유사도 점수: 7.0601
내담자 정보: {'거주지역': '시흥시', '국적': '필리핀', '체류자격': 'E-9', '추가정보': '\n필리핀 국적 노동자\n한국에서 약 11년간 근무\nE-7비자 신청 과정 중 이중 취업 문제로 신청 탈락\n다른 회사에서 허위로 세무 신고된 급여로 인해 E-7비자 신청이 좌절됨\n'}
해결 방법: 

이중 취업 문제로 E-7 비자 신청이 좌절되었으나, 세무서를 통한 기록 삭제가 불가하여 출입국 사무소에 벌금 200만원을 지불하는 방안을 고려했으나, 비자 만료 이전 신청이 불가하여 귀국을 결정하였습니다. 귀국 준비를 위한 병원 진료, 노동부, 국민연금공단, 출국 만기보험 수속 절차를 지원하였고, 인천 공항까지 동반하여 귀국 절차를 안내하였습니다. 귀국 후 국민연금과 출국만기보험을 수령하였으며, 필리핀에서 재정착 중입니다.


3번째 유사 사례:
유사